<a href="https://colab.research.google.com/github/verya1361/AzadiDNSPanel/blob/main/PDF_Translator_Februaru_08_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install

# Settings

In [52]:

#@title فراخوانی کلید دسترسی
import google.generativeai as genai

GOOGLE_API_KEY = "" #@param {type:"string"}
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:

#@title آپلود فایل PDF
from google.colab import files
import PyPDF2
import io

# آپلود فایل
uploaded = files.upload()

if not uploaded:
    raise ValueError("هیچ فایلی آپلود نشد. لطفاً یک فایل PDF آپلود کنید.")

filename = list(uploaded.keys())[0]

# بررسی فرمت فایل
if not filename.lower().endswith('.pdf'):
    raise ValueError("فرمت فایل اشتباه است. لطفاً یک فایل PDF آپلود کنید.")

# خواندن فایل PDF
pdf_reader = PyPDF2.PdfReader(io.BytesIO(uploaded[filename]))
num_pages = len(pdf_reader.pages)
print(f"تعداد کل صفحات: {num_pages}")

In [ ]:

#@title تنظیم محدوده صفحات
start_page = 1  #@param {type:"slider", min:1, max:1000, step:1}
end_page = 72  #@param {type:"slider", min:1, max:1000, step:1}

# بررسی محدوده صفحات
if start_page < 1 or end_page > num_pages:
    raise ValueError(f"شماره صفحات باید بین 1 و {num_pages} باشد.")

print(f"ترجمه از صفحه {start_page} تا صفحه {end_page} انجام خواهد شد")

#Translate

In [53]:
#@title ترجمه به فارسی
#@markdown ### `در صورت مشاهده خطا از سمت گوگل جمنای ، مقدار نوار اسلایدر زیر را با عدد بالاتری تست کنید یا توکن را تغییر دهید`

sleep_time = 3 #@param {type:"slider", min:2, max:5, step:1}

from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re

def clean_text(text):
    # محدود کردن طول متن
    text = text[:15000]  # محدود کردن به 15000 کاراکتر
    text = ''.join(char for char in text if char == '\n' or char == '\t' or (ord(char) >= 32))
    text = text.replace('\x00', '')
    return text

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=2, min=4, max=20))
def translate_page_with_retry(text):
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    prompt = f"""متن زیر را به فارسی عامیانه و روان ترجمه کن. نکات مهم:

1. فقط ترجمه خالص و عامیانه، بدون هیچ توضیح اضافه
2. به شدت مهمه که تا معنای واقعی جملات رو درک نکنی حق ترجمه نداری
3. استفاده از زبان محاوره‌ای و روزمره
4. ترجمه نباید بی ربط باشه و سعی کن تا جایی که ممکنه حرفه ای عمل کنی

متن برای ترجمه:
{text}"""
    time.sleep(sleep_time + 2)  # افزایش زمان انتظار
    response = model.generate_content(prompt)
    return clean_text(response.text)

# ایجاد سند Word جدید
doc = Document()

# شروع ترجمه صفحات
with tqdm(total=end_page-start_page+1, desc="ترجمه صفحات") as pbar:
    for page_num in range(start_page-1, end_page):
        try:
            page_text = pdf_reader.pages[page_num].extract_text().strip()

            # بررسی خالی بودن صفحه
            if len(page_text) < 50:
                print(f"\nصفحه {page_num+1} خالی یا غیر قابل پردازش است - رد شد")
                pbar.update(1)
                continue

            translated_page = translate_page_with_retry(clean_text(page_text))

            # اضافه کردن شماره صفحه
            heading = doc.add_paragraph(f"=== صفحه {page_num+1} ===")
            heading.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

            # اضافه کردن متن ترجمه شده
            paragraph = doc.add_paragraph(translated_page)
            paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

            pbar.update(1)
            time.sleep(2)  # استراحت بین صفحات

        except Exception as e:
            print(f"\nخطا در صفحه {page_num+1}: {str(e)}")
            time.sleep(5)  # استراحت بیشتر در صورت خطا
            continue

# ذخیره فایل
doc.save(f"{filename}_translated.docx")
print("\nترجمه با موفقیت به پایان رسید!")

NameError: name 'end_page' is not defined

اگر هنگام ترجمه ، اخطار زیر نمایش داده شد ولی پروسه همچنان در حال پیشروی بود اخطار را نادیده بگیرید

WARNING:tornado.access ....

و اگر غیر از این ، خطایی مشاهده کردید بهمون اطلاع بدین

**<font color='blue'> `-----------------------------------` </font>**

In [54]:
#@title Docx دانلود با فرمت
from google.colab import files
files.download(f"{filename}_translated.docx")

FileNotFoundError: Cannot find file: nn.pdf_translated.docx

In [55]:

#@title PDF دانلود با فرمت
!soffice --headless --convert-to pdf "{filename}_translated.docx"
from google.colab import files
files.download(f"{filename}_translated.pdf")

Error: source file could not be loaded


FileNotFoundError: Cannot find file: nn.pdf_translated.pdf